In [1]:
!pip install git+https://github.com/vcoscrato/uncertain.git#egg=uncertain

  Cloning https://github.com/vcoscrato/uncertain.git to /tmp/pip-install-n9ypmb0n/uncertain
  Running command git clone -q https://github.com/vcoscrato/uncertain.git /tmp/pip-install-n9ypmb0n/uncertain
  Created wheel for uncertain: filename=uncertain-0.0.1-cp36-none-any.whl size=23763 sha256=e57875da14866d7f6fa250b14a8c8254225444cd2fa271d1518f80f88eda0ecf
  Stored in directory: /tmp/pip-ephem-wheel-cache-sqtdfxxs/wheels/8d/54/55/f301254d9c7ee96a6c007b4d92a1e9b8f4cfe6300dcfad2455
Successfully built uncertain


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Mounted at /content/drive
/content/drive/My Drive


In [3]:
import torch
from pickle import load
from pandas import factorize

from uncertain import Interactions
from uncertain.models import FunkSVD, CPMF, OrdRec, MultiModelling
from uncertain.models import LinearUncertainty, CVUncertainty, PlugIn

with open('Netflix/netflix.pkl', 'rb') as f:
    data = list(load(f))

torch.manual_seed(0)
shuffle = torch.randperm(len(data[0]))

for i in range(3):
  data[i] = data[i][shuffle]
interactions = torch.tensor((data[0], data[1])).T.cuda()
ratings = torch.tensor((data[2])).cuda()

cutoff_train = int(0.6 * len(data[0]))
cutoff_val = cutoff_train + int(0.2 * len(data[0]))

train_idx = slice(None, cutoff_train)
validation_idx = slice(cutoff_train, cutoff_val)
test_idx = slice(cutoff_val, None)

In [4]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.fit(train, val)

with open('Netflix/results.txt', 'a') as f:
  print('baseline = ', baseline.evaluate(test, train), '\n', file=f)

Previous training file encountered. Loading and resuming training.
Epoch 1 loss - Train: 0.6007446089967466, Validation: 0.7650690060942921 - This is the lowest validation loss so far
Epoch 2 loss - Train: 0.5778748313103178, Validation: 0.7694845564329802 - Validation loss did not improve. Reducing learning rate.
Epoch 2 loss - Train: 0.5254263128886337, Validation: 0.7522477187327484 - This is the lowest validation loss so far
Epoch 3 loss - Train: 0.5129627903588341, Validation: 0.7571521629741536 - Validation loss did not improve. Reducing learning rate.
Epoch 3 loss - Train: 0.48599594055727036, Validation: 0.7526401058951421 - Validation loss did not improve. Ending training.


In [5]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

user_support = PlugIn(baseline, LinearUncertainty(None, -train.get_user_support()))

with open('Netflix/results.txt', 'a') as f:
  print('usup = ', user_support.evaluate(test, train), '\n', file=f)

In [6]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

item_support = PlugIn(baseline, LinearUncertainty(None, -train.get_item_support()))

with open('Netflix/results.txt', 'a') as f:
  print('isup = ', item_support.evaluate(test, train), '\n', file=f)

In [4]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

item_variance = PlugIn(baseline, LinearUncertainty(None, train.get_item_variance()))

with open('Netflix/results.txt', 'a') as f:
  print('ivar = ', item_variance.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/funksvdcv.pth', 'use_cuda': True}
funksvdcv = CVUncertainty(recommender=baseline, params=MF_params)
funksvdcv.fit(train, val)
funksvdcv = PlugIn(baseline, funksvdcv.uncertainty)

with open('Netflix/results.txt', 'a') as f:
  print('funksvdcv = ', funksvdcv.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

MF_params = {'embedding_dim': 0, 'l2': 0, 'learning_rate': 2,
             'batch_size': 512, 'path': 'Netflix/Fitted/biascv.pth', 'use_cuda': True}
biascv = CVUncertainty(recommender=baseline, params=MF_params)
biascv.fit(train, val)
biascv = PlugIn(baseline, biascv.uncertainty)

with open('Netflix/results.txt', 'a') as f:
  print('biascv = ', biascv.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

resample = MultiModelling(base_model=baseline, n_models=5, type='Resample')
resample.fit(train, val)

with open('Netflix/results.txt', 'a') as f:
  print('resample = ', resample.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Netflix/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

ensemble = MultiModelling(base_model=baseline, n_models=5, type='Ensemble')
ensemble.fit(train, val)

with open('Netflix/results.txt', 'a') as f:
  print('ensemble = ', ensemble.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

CPMF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
               'batch_size': 512, 'path': 'Netflix/Fitted/CPMF.pth', 'use_cuda': True}
cpmf = CPMF(**CPMF_params)
cpmf.fit(train, val)

with open('Netflix/results.txt', 'a') as f:
  print('cpmf = ', cpmf.evaluate(test, train), '\n', file=f)

In [ ]:
factorized_ratings = factorize(ratings.cpu().numpy(), sort=True)
ratings_ = factorized_ratings[0].cuda()
labels = torch.tensor(factorized_ratings[1]).cuda()

train = Interactions(interactions[train_idx], ratings_[train_idx])
val = Interactions(interactions[validation_idx], ratings_[0][validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

OrdRec_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 10,
                 'batch_size': 512, 'path': 'Netflix/Fitted/OrdRec.pth', 'use_cuda': True}
ordrec = OrdRec(labels, **OrdRec_params)
ordrec.fit(train, val)

with open('Netflix/results.txt', 'a') as f:
  print('ordrec = ', ordrec.evaluate(test, train), '\n', file=f)